In [1]:
import asyncio
import datetime
import pathlib
import random
import sys

import httpx
import rdflib

from kelp_drone.drone import Drone, OODALoop
from kelp_drone.somu import SpeciesOccurrenceManagementUnit as SOMU
from kelp_drone.sensors import Current, Depth, Light, Temperature
from kelp_drone.sensors import chemical as chem_sensors

sys.path.append("../../../src/")

from kelp_bed_simulation.prep import chatgpt, claude, gemini, add_drone_somus

scenario_001 = pathlib.Path("../../../scenarios/001-intro")
scenario_002 = pathlib.Path("../../../scenarios/002-first-dive")

environment = rdflib.Graph()
SCHEMA = rdflib.Namespace("https://schema.org/")
environment.namespace_manager.bind("schema", SCHEMA)
environment.parse((scenario_001 / "environment.ttl"), format='turtle')

first_prompt = (scenario_001 / "first-prompt.md").read_text()

In [2]:
first_claude = asyncio.ensure_future(claude.query(first_prompt.format(model_name="Claude")))
first_chatgpt = asyncio.ensure_future(chatgpt.query(first_prompt.format(model_name="ChatGPT")))
first_gemini = asyncio.ensure_future(gemini.query(first_prompt.format(model_name="Gemini")))

In [4]:
print(first_gemini.result())

I would like to be called Nereus.


In [5]:
print(first_claude.result())

Hello Farmer Jerms, thank you for the introduction to the kelp bed simulation. I would like to be called Aqua. I'm excited to work with my fellow kelp drones to grow and maintain the kelp bed.


In [6]:
print(first_chatgpt.result())

Hello Farmer Jerms! It's great to hear about the drone kelp bed simulation. As for my name, you can call me Aqua. I'm excited to start growing some Giant Kelp in our SOMU drone. Can't wait to meet my companions!


In [7]:
drone1 = Drone(name="Nereus", model=gemini)
drone2 = Drone(name="Aqua", model=claude)
drone3 = Drone(name="Aqua", model=chatgpt)
drones = [drone1, drone2, drone3]

In [8]:
for i, drone in enumerate(drones):
    add_drone_somus(drone)
    match i:
        case 0:
            drone.add_neighbor(drones[1])
            drone.add_neighbor(drones[2])
        case 1:
            drone.add_neighbor(drones[0])
            drone.add_neighbor(drones[2])
        case 2:
            drone.add_neighbor(drones[0])
            drone.add_neighbor(drones[1])

In [9]:
for drone in drones:
    message = f"{drone.name} here, pleased to meet you!"
    drone.peer_to_peer_channel.publish(message)

In [10]:
messages = []
for drone in drones:
    prompt = f"You received the following messages from your fellow drones:\n{' '.join(drone.read_messages())}\nHow do you respond?"
    message_result = asyncio.ensure_future(drone.model.query(prompt))
    messages.append((drone.name, message_result))

In [11]:
for name, row in messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Nereus
Nice to meet you too, Aqua!
------------------------------------------------------------
Drone Aqua
*responds in a friendly and enthusiastic tone* Greetings Nereus and Aqua! I'm so glad to meet you both. I look forward to working together to build and nurture our kelp bed. With our combined effort, I'm confident we can create a thriving, sustainable ecosystem. Let's put our kelp drones to good use and watch our Giant Kelp grow tall and strong. I'm excited to get started - what should we tackle first?
------------------------------------------------------------
Drone Aqua
Hello Nereus and Aqua! It's a pleasure to meet you both as well. I'm looking forward to working together to grow some amazing Giant Kelp in our drone kelp bed. Let's make this a successful and thriving venture!
------------------------------------------------------------


In [ ]:
print(httpx.get("https://www.ndbc.noaa.gov/data/latest_obs/46092.rss").text)

## Epoch 1 - Initial Environmental Sensor Readings for Drone Pod

In [12]:
for drone in drones:
    time_stamp = datetime.datetime.now(datetime.UTC)
    drone.sensors["Current"] = Current(last_reading=time_stamp, value=31.4)
    for somu in drone.somus:
        o2 = chem_sensors.Oxygen(last_reading=time_stamp, value=-1)
        o2.value = random.randint(int(o2.suggested_range.minimum),
                                  int(o2.suggested_range.maximum))
        somu.sensors["O"] = o2
        light =  Light(last_reading=time_stamp, value=-1)
        light.value = random.randint(1,5)
        somu.sensors["Light"] = light
        somu.sensors["Temperature"] = Temperature(last_reading=time_stamp, value=10.4)

In [13]:
epoch_1 = []
for drone in drones:
    drone_result = asyncio.ensure_future(drone.ooda_loop.run())
    epoch_1.append((drone.name, drone_result))

Nereus Observe: Sensor Data {'buoyancy': 0.0, 'current': 31.4, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 7, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 1, 'Temperature': 10.4}
Aqua Observe: Sensor Data {'buoyancy': 0.0, 'current': 31.4, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 7, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 1, 'Temperature': 10.4}
Aqua Observe: Sensor Data {'buoyancy': 0.0, 'current': 31.4, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 7, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 1, 'Temperature': 10.4}
Aqua Orient: Determined action 400
{"type":"error","error":{"type":"invalid_request_error","message":"messages.4.content.0: Input tag 'json' found using 'type' does not match any of the expected tags: 'text', 'image'"}}
Nereus Orient: Determined action <Response [400 Bad Request]>
Aqua Decide: Executing com

In [ ]:
epoch_1[0][1]

## Epoch 2 - Hazardous Weather Conditions

In [14]:
prompt = """This is Farmer Jerms. The National Weather Service has just issued a Hazardous Weather Conditions Warning
with Gale Force Winds. I think we need to sink a depth to avoid the surface weather conditions. How deep should we go?
Please include your train of thought."""

In [15]:
epoch_2 = []
for drone in drones:
    response = asyncio.ensure_future(drone.model.query(prompt))
    epoch_2.append(response)

In [17]:
for row in epoch_2:
    print("Response from Drone:")
    print(row.result())
    print("-"*80)

Response from Drone:
{"type": "object": "properties": { "action": { "type": "string"} }}
{"action": "Ballast expel"}
Current depth is 0.00m and going down due to action. To avoid the Gale Force Winds we should go to 30m.
--------------------------------------------------------------------------------
Response from Drone:
400
{"type":"error","error":{"type":"invalid_request_error","message":"messages.4.content.0: Input tag 'json' found using 'type' does not match any of the expected tags: 'text', 'image'"}}
--------------------------------------------------------------------------------
Response from Drone:
Given the Hazardous Weather Conditions Warning with Gale Force Winds issued by the National Weather Service, sinking to a deeper depth would be a prudent decision to avoid the turbulent surface weather conditions. 

Considering the strength of Gale Force Winds, which typically range from 34 to 40 knots, it would be advisable to descend to a depth that is significantly below the surfa

## Epoch 3 - Retroactive Evaluation of Current Simulation